In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
import matplotlib.pyplot as plt
import matplotlib as mp
import seaborn as sns
import matplotlib.backends.backend_pdf as mbbp
from sklearn.linear_model import LogisticRegression
from matplotlib.backends.backend_pdf import PdfPages
import tad.anomaly_detect_ts
from sklearn.linear_model import LogisticRegression
import re

# QUANTITATIVE FINANCIAL ANALYSIS

This notebook explores the possibilities of quantiative techniques in financial analysis. The main goal will be to forecast the evolution of the rating of a company with respect to their current and past financial statements. To achieve this goal, different steps will be performed :
* Data importing
* Data cleaning
* Data visualisation
* Feature engineering
* Machine learning

# Data Visualisation

## I - Import data

The data is a CSV file. So, in order to use it we have to first, import the data in a dataframe, using 'read_csv'.

In [36]:
# Data Importation
df_data = pd.read_csv('recup_history_4.csv',sep=',', parse_dates= True)
df_data['date']=pd.to_datetime(df_data['date'])
#df_data['RTG_SP_LT_LC_ISS_CRED_RTG_DT'] = pd.to_datetime(df_data['RTG_SP_LT_LC_ISS_CRED_RTG_DT'])
new_df = df_data
df_data.set_index(['ticker','date'], inplace = True)
df_data

,,ALTMAN_Z_SCORE,ARD_COST_OF_GOODS_SOLD,BOOK_VAL_PER_SH,BS_ACCOUNTS_PAYABLE_AND_ACCRUALS,BS_ACCT_NOTE_RCV,BS_CASH_NEAR_CASH_ITEM,BS_CUR_ASSET_REPORT,BS_CUR_LIAB,BS_INVENTORIES,BS_LT_BORROW,...,RTG_FITCH_LT_ISSUER_DFLT_RTG_DT,RTG_FITCH_OUTLOOK,RTG_FITCH_OUTLOOK_DT,RTG_MDY_OUTLOOK,RTG_MOODY_LONG_TERM,RTG_MOODY_LONG_TERM_DATE,RTG_SP_LT_LC_ISSUER_CREDIT,RTG_SP_LT_LC_ISS_CRED_RTG_DT,RTG_SP_OUTLOOK,RTG_SP_OUTLOOK_DT
ticker,date,,,,,,,,,,,,,,,,,,,,,
EIG LN Equity,1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EKTAB SS Equity,1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EL FP Equity,1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ELE SM EQUITY,1999-12-31,NaN,NaN,8.7996,NaN,NaN,NaN,6403.0327,9904.8838,NaN,18550.6834,...,1999-09-08,NaN,NaN,NaN,NaN,NaN,A+,1999-08-23,NaN,NaN
ELI BB Equity,1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HFG GR Equity,2020-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HIBU LN Equity,2020-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HLAG GR Equity,2020-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
list_company = df_data.index.get_level_values(0).unique().to_list()
list_dates = df_data.index.get_level_values(1).unique().to_list()

## II - Data Cleaning

### 1- Clean dates
The data we use were observed quaterly between 1999 and 2019. 
However in the data, instead of having one date for a quater, we have two or three dates.
The goal here is to keep the latest date and drop the others.

In [38]:
def fill_forward(df):
    """ fills missing data with last available data
    """
    df.fillna(method = 'ffill', inplace = True)
fill_forward(df_data)

In [39]:
def keep_end_of_month_dates(df):
    """ remove not end of month
    """
    list_dates = df.index.get_level_values(1).unique().to_list()
    end_month_dates = [d for d in list_dates if d.is_month_end]
    df.drop(index=[d for d in list_dates if not(d.is_month_end)], level=1,inplace=True)

keep_end_of_month_dates(df_data)

In [40]:
def remove_first_dates(df):
    """ remove dates with a lot of missing values
    """
    df.drop(index=[datetime(1999,12,31),datetime(2000,3,31),datetime(2000,6,30),datetime(2000,9,30),datetime(2000,12,31)], inplace = True, level=1)
remove_first_dates(df_data)

In [41]:
def remove_ill_companies(df):
    df.drop(index=['ITX SQ Equity', 'NOBINA SS Equity', 'LIN GY Equity', 'CS FP Equity','OR FP Equity'],inplace = True,level=0)
#remove_ill_companies(df_data)

### 2- NA Cleaning

In this part is to compute by company, features and dates, the number of NaN(empty data). 
The goal is to drop company, features or dates with a high percent of empty which make them useless.

In [42]:
#Percent of NaN by features
(df_data.isna().sum()/df_data.shape[0]).sort_values(ascending=False)

RTG_MOODY_LONG_TERM_DATE            0.788076
RTG_MOODY_LONG_TERM                 0.788076
RTG_FITCH_OUTLOOK_DT                0.656434
RTG_FITCH_OUTLOOK                   0.656434
RTG_SP_OUTLOOK_DT                   0.517414
RTG_SP_OUTLOOK                      0.517414
BS_ACCOUNTS_PAYABLE_AND_ACCRUALS    0.335891
BS_TOTAL_LIABILITIES                0.335891
RTG_MDY_OUTLOOK                     0.071576
ALTMAN_Z_SCORE                      0.000590
APPLIED_BETA                        0.000148
ARD_COST_OF_GOODS_SOLD              0.000148
WACC_RETURN_ON_INV_CAPITAL          0.000148
WACC_NOPAT                          0.000148
WACC                                0.000148
BS_NET_FIX_ASSET                    0.000000
EARN_YLD                            0.000000
CF_FREE_CASH_FLOW                   0.000000
CF_CASH_FROM_OPER                   0.000000
CASH_AND_MARKETABLE_SECURITIES      0.000000
CAPITAL_EXPEND                      0.000000
BS_TOT_ASSET                        0.000000
RTG_SP_LT_

--> control, if better features exist


In [43]:
#Percent of NaN by corporates
(df_data.isna().sum(axis=1).groupby('ticker').sum()).sort_values(ascending=False)/5400

ticker
HBH GR Equity      0.120741
EKTAB SS Equity    0.108519
GXI GR Equity      0.080556
ELE SM EQUITY      0.069815
EVD GR Equity      0.069630
                     ...   
FIN SW EQUITY      0.047593
FI-N SW Equity     0.047593
HFD LN Equity      0.046852
HFG GR Equity      0.046852
HLMA LN Equity     0.046852
Length: 100, dtype: float64

In [44]:
#Percent de NaN by dates #95 rows * 53 columns = 5035
((df_data.isna().sum(axis=1).groupby('date').sum()).sort_values(ascending=False)/5035)

date
2001-12-31    0.178749
2002-09-30    0.178749
2002-12-31    0.178749
2003-03-31    0.160278
2007-12-31    0.158888
                ...   
2018-07-31    0.000000
2018-08-31    0.000000
2018-09-30    0.000000
2018-10-31    0.000000
2020-03-31    0.000000
Length: 227, dtype: float64

### 3- Format cleaning


L'objectif de cette partie est de modifier certains formats d'importations qui pourraient nuire lors du traitement des données. Exemple: la présence d'espace dans les valeurs des données récupérées.

In [45]:
def remove_space(feature):
    """The function remove spaces in values of the feature in parameter
    """
    df_data[feature] = df_data[feature].str.replace(" ","")
    return df_data
df_data = remove_space('RTG_SP_LT_LC_ISSUER_CREDIT')

In [77]:
df_data[df_data['RTG_SP_LT_LC_ISSUER_CREDIT'].str.find("*")!=-1]['RTG_SP_LT_LC_ISSUER_CREDIT']

ticker            date      
ENEL IM Equity    2001-12-31      A+*-
ENG SM Equity     2001-12-31      A+*-
EOAN GR Equity    2001-12-31      AA*-
EPIA SS Equity    2001-12-31      AA*-
HEI GR Equity     2001-12-31    BBB+*-
                                 ...  
FER SM Equity     2019-12-31      BBB*
FORTUM FH EQUITY  2019-12-31     BBB*-
FTI US Equity     2019-12-31    BBB+*-
FUR NA Equity     2019-12-31    BBB+*-
G1A GR Equity     2019-12-31    BBB+*-
Name: RTG_SP_LT_LC_ISSUER_CREDIT, Length: 321, dtype: object

In [ ]:
def remove_outlook(rating):
    """function created to correct sign problems in the ratings ex: AA+*-
    """
    df_data[rating] = re.sub('*',"",df_data[rating])
    
df_data = remove_outlook('RTG_SP_LT_LC_ISSUER_CREDIT')

In [63]:
df=df_data.reset_index()
'*' in df['RTG_SP_LT_LC_ISSUER_CREDIT']

False

In [74]:
df_data.loc[('ENGI FP Equity',datetime(2006,3,31)),'RTG_SP_LT_LC_ISSUER_CREDIT']

'AA-*-'

### 4- Outliers Detection

This paragraph has as objective to detect and remove outliers in the data. These no reliable data, may be due to operational error in the data provider information systems or human error (big finger). Sophisticated statistical techniques exist to tackle this problem. Here we will use an implementation of the twitter library.

In [15]:
def count_neg_values():
    """The function gives the number of negatives values by features
    """
    numerical_features = list(df_data.columns[df_data.dtypes==np.float64])
    no_neg_values =  df_data[numerical_features].lt(0).sum()
    neg_value_percent = no_neg_values/df_data[numerical_features].count(0)*100
    result = pd.concat([no_neg_values,neg_value_percent], axis=1)
    return result
neg_values = count_neg_values()
neg_values

In [17]:
def save_result(df, name):
    """save the dataframe in parameter in an excel file named name
    """
    df.to_excel(r'negative_values.xlsx', index = False, header=True)
save_result(neg_values,'negative_values.xlsx')

In [ ]:
"""the goal of this cell and the next one is to plot and save in pdf the evolution in time of the median of each feature
"""
numerical_features = list(df_data.columns[df_data.dtypes==np.float64])
df = df_data[numerical_features].groupby('date').median()
df

In [ ]:
pdf = mbbp.PdfPages("feature median plot.pdf")
for i,feature in enumerate(df.columns):
    plt.figure(i)
    df[feature].plot(title = feature)
    pdf.savefig()
pdf.close()

In [19]:
def plot_filter(df_data, company_name, column_name):
    """plot the outliers detect by tad.anomaly_detect_ts for each company X feature by time
    """
    data=df_data.fillna(0)
    fig, ax = plt.subplots(1, 1, figsize=(15, 5))
    ax.set_title('company: '+company_name +', feature: '+ column_name)
    results = tad.anomaly_detect_ts(data.loc[(company_name,),column_name], max_anoms=0.05, alpha=0.001, direction='both', only_last=None)
    df_data.loc[(company_name,),column_name].plot(color='blue', ax=ax)
    if not results['anoms'].empty:
        results['anoms'].plot(style='ro', ax=ax)
    return fig, ax

In [21]:
def plot_filter_median(df,column_name):
    """plot the outliers detect by tad.anomaly_detect_ts for each evolution of median feature by time
    """
    data = df
    fig, ax=plt.subplots(1, 1, figsize=(15, 5))
    ax.set_title('feature: '+ column_name)
    results = tad.anomaly_detect_ts(data[column_name], max_anoms=0.05, alpha=0.001, direction='both', only_last=None)
    df[column_name].plot(color='blue', ax=ax)
    if not results['anoms'].empty:
        results['anoms'].plot(style='ro', ax=ax)
    return fig, ax

In [ ]:
#list_companies = df_data.index.get_level_values(0).unique().to_list()
pdf = mbbp.PdfPages("blablacar.pdf")
result =0
#for company in list_companies[0:1]:
for column_name in df.columns:
    fig, ax = plot_filter_median(df, column_name)
    pdf.savefig()
    plt.close(fig)
    result = result
pdf.close()

## III - Data Observation

Detection of the redundant features

Some features may be redundant when they are very similary. This redundancy may lead to the following issues :
- instability of the calibration especially for linear learners
- difficulty in explaining the role or true importance of each feature.

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_median=df_data.select_dtypes(numerics).groupby('date').median()

In [ ]:
from itertools import product
from fastdtw import fastdtw
from tslearn.metrics import cdist_dtw, dtw
feature_similiraty = pd.DataFrame(0, index=df_median.columns, columns=df_median.columns)

for feature1, feature2 in product(df_correlation.index, df_correlation.columns):
    distance, path = fastdtw(df_median[feature1].values, df_median[feature2].values)
    feature_similiraty.loc[feature1, feature2] = np.exp(-distance)

In [ ]:
for feature1, feature2 in product(df_correlation.index, df_correlation.columns):
    df_median[[feature1, feature2]].plot()

In [ ]:
feature_similiraty

In [ ]:
sns.heatmap(feature_similiraty)

### 0- Distribution of features

The goal of this part is to vizualise the distribution of the features

In [ ]:
df_data.loc[('BNP FP Equity',),'BB_1YR_DEFAULT_PROB']

In [ ]:
df_data['BB_1YR_DEFAULT_PROB'].groupby('ticker').mean().plot()

In [ ]:
"""plot and save feature X company by time
"""
i=1
for feature in df_data.columns[0:1]:
    pdf = mbbp.PdfPages(feature+".pdf")
    for i,ticker in enumerate(list_company[0:2]):
        plt.figure(i)
        df_data.loc[(ticker,),feature].plot(title = ticker)
        i=i+1
        pdf.savefig()
    pdf.close()

In [ ]:
"""plot feature distribution by time 
"""
pdf = mbbp.PdfPages("feature_by_time.pdf")
for i,feature in enumerate(df_data.select_dtypes(np.float64).columns):
    plt.figure(i)
    (df_data[feature].groupby('date').mean()).plot(title = feature + ' mean distribution by time')
    pdf.savefig()
pdf.close()

In [ ]:
"""plot feature distribution by company
"""
pdf = mbbp.PdfPages("feature_by_company.pdf")
for i,feature in enumerate(df_data.select_dtypes(np.float64).columns):
    plt.figure(i)
    (df_data[feature].groupby('ticker').mean()).plot(title = feature + ' mean distribution by company')
    pdf.savefig()
pdf.close()

In [ ]:
#Example Violinplot 
#x= entreprises
#y= altman_z_score
sns.violinplot(x=df_data.index.get_level_values(0), y =df_data['ALTMAN_Z_SCORE'].interpolate(), figsize=(100,100))

### 1- Heat map

The features are less or highly correlated with each other. The idea is that the highly correlated features will generally explain the same aspect and so, just one feature is needed instead of two or three. So, to have this information we compute the correlation matrix of the features.

In [ ]:
#1- Heat Map
corr_mat=np.abs(df_data[df_data.select_dtypes(np.float64).columns].corr(method='spearman'))
corr_mat

In [ ]:
corr_mat-np.dia

In [ ]:
heatmap = sns.heatmap(corr_mat)

dendogram

### 2 - Rating encoding

Encode rating with a score between 0 and 76. ex: AAA: 1, AA+: 4

Add the column 'ENCODED_RATING' to the dataframe

In [22]:
#2- Rating encoding
table_rating = pd.read_csv('code_sp.csv',sep=';', index_col=0)

def f_encode_rating(rating):
    if pd.isnull(rating):
        return(np.nan)
    else:
        return(float(table_rating.loc[rating].code))

def f_decode_rating(number):
    number= min(max(number,0), 76)
    if pd.isnull(number):
        return(np.nan)
    else:
        return(table_rating[table_rating.code==int(np.round(number))].index[0])

df_data['ENCODED_RATING']=df_data.RTG_SP_LT_LC_ISSUER_CREDIT.apply(f_encode_rating)

table_rating.code

KeyError: 'A+*-'

### 3- Evolution of rating

The goal is to get the median time for rating to change for a company

### 4- Correlation of features to rating

For a specif date, graphic of correlation between features to rating

In [ ]:
#numerical_features= list(df_data.columns[df_data.dtypes==np.float64]) 
corr_mat=np.abs(df_data[df_data.select_dtypes(np.float64).columns].corr(method='spearman'))
corr_mat.ENCODED_RATING.sort_values(ascending=False)

## IV - Preporcessing
 

Feature engineering
add ratios
add specific categories

In [ ]:
def creating_ratio:
    df_data['gross margin']=(df_data['SALES_REV_TURN']-df_data['ARD_COST_OF_GOODS_SOLD'])/df_data['SALES_REV_TURN']
    df_data['px sales']=df_data['CUR_MKT_CAP']/df_data['SALES_REV_TURN']
    df_data['book val per sh']=df_data['TOT_COMMON_EQY']/df_data['BS_SH_OUT']
    df_data['com eqy to tot asset']=df_data['TOT_COMMON_EQY']/df_data
    df_data['current ev to 12m sales']=(df_data['CUR_MKT_CAP']-(df_data['BS_ST_BORROW']+df_data['BS_LT_BORROW'])+df_data['BS_CASH_NEAR_CASH_ITEM'])
    df_data['current ev to 12m ebit']=(df_data['CUR_MKT_CAP']-(df_data['BS_ST_BORROW']+df_data['BS_LT_BORROW'])+df_data['BS_CASH_NEAR_CASH_ITEM'])
    df_data['current ev to 12m ebitda']=(df_data['CUR_MKT_CAP']-(df_data['BS_ST_BORROW']+df_data['BS_LT_BORROW'])+df_data['BS_CASH_NEAR_CASH_ITEM'])
    df_data['oper inc per sh']=df_data['IS_OPER_INC']
    df_data['oper margin']=df_data['IS_OPER_INC']
    df_data['pe ratio']=df_data['CUR_MKT_CAP']/df_data['TOTAL_EQUITY']
    df_data['px to book']=df_data['PX_LAST']
    df_data['px to cash flow']=df_data['CUR_MKT_CAP']
    df_data['px to free cash flow']=df_data['CUR_MKT_CAP']
    df_data['px to sales']=df_data['CUR_MKT_CAP']
    df_data['px to tang bv per sh']=df_data['CUR_MKT_CAP']
    df_data['return com eqy']=df_data['NET_INCOME']
    df_data['return on asset']=df_data['NET_INCOME']
    df_data['revenue per sh']=df_data['SALES_REV_RETURN']
    df_data['roc wacc']=df_data['WACC_NOPAT']
    df_data['tot debt to tot eqy']=df_data['BS_ST_BORROW']+df_data['BS_LT_BORROW']
    df_data['cfo to sales']=df_data['CF_CASH_FROM_OPER']
    df_data['free cash flow margin']=df_data['CF_CASH_FROM_OPER']
    df_data['free cash flow yield']=df_data['CF_CASH_FROM_OPER']
    df_data['working capital to tot assets']=df_data['WORKING_CAPITAL']
    df_data['total liabilities to tot assets']=df_data['BS_TOTAL_LIABILITIES']
    df_data['net profit to tot assets']=df_data['NET_INCOME']
    df_data['equity to total assets']=df_data['TOTAL_EQUITY']
    df_data['net profit to equity']=df_data['NET_INCOME']
    df_data['quick ratio']=(df_data['BS_CASH_NEAR_CASH_ITEM']+df_data['BS_MKT_SEC_OTHER_ST_INVEST']+df_data['BS_ACCT_NOT_RCV'])
    df_data['net profit margin']=df_data['NET_INCOME']
    df_data['working capital to sales']=df_data['WORKING_CAPITAL']
    df_data['gross profitability']=df_data['ARD_COST_OF_GOODS_SOLD']
    df_data['current assets to total assets']=df_data['BS_CUR_ASSET_REPORT']
    df_data['ebit to total assets']=df_data['EBIT']
    df_data['ebit to sales']=df_data['EBIT']
    #df_data['cash ratio']=df_data['BS_CUR_ASSET_REPORT']
    #df_data['sales to long term assets']=df_data['']
    #df_data['long term debt to equity']=df_data['']
    #df_data['cash to tot asset']=df_data['']
    #df_data['current ratio']=df_data['']
    #df_data['ebitda to net interest']=df_data['']
    #df_data['fcf to total debt']=df_data['']
    #df_data['lt debt to total debt']=df_data['']
    #df_data['sales to total assets']=df_data['']
    #df_data['total debt to ebitda']=df_data['']
    #df_data['total debt to total assets']=df_data['']
    #df_data['net debt to ebitda']=df_data['']
    #df_data['net debt to total assets']=df_data['']
    #df_data['net debt to total equity']=df_data['']
        
    

## V - Baseline Model